# INCOMPLETE !!!

In [ ]:
import albumentations as A # to make holes 
from torchvision.transforms import v2 # for other transforms 

import torch 
import torch.nn as nn 
import torch.nn.functional as F 

from PIL import Image 

seed = 10 
torch.manual_seed(seed) 
torch.cuda.manual_seed(seed) 
torch.cuda.manual_seed_all(seed) 

c:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
cut_tfm = A.CoarseDropout(num_holes_range=(1,3), p=1)
tv_tfms = v2.Compose([
    v2.ToTensor(), 
    v2.ToDtype(torch.float32), 
    v2.RandomHorizontalFlip(p=0.5), 
    v2.RandomAffine(degrees=(-10, 10), scale=(0.9, 1.1))
])



c:\Python\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [5]:
from torchvision.datasets import MNIST 

train_ds = MNIST('./data/', train=True, download=True, target_transform=tv_tfms)

def train_preprocess(torch_img, device=device): 
    return torch.tensor(cut_tfm(image=torch_img.numpy())['image'], device=device) 



NameError: name 'tv_tfms' is not defined

conv, then deconv 

In [ ]:
class ImageVAE_enc(nn.Module): 
    def __init__(self, d=32, hidden_layer_size=100):
        super(ImageVAE_enc, self).__init__()

        # conv (1 input channel) to a small layer inside 
        # 28x28 (MNIST dataset)
        self.conv1 = nn.Conv2d(1, d, 4, 2, 3, dtype=torch.float32) # 16x16 
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1, dtype=torch.float32) # 8x8 
        self.conv2_bn = nn.BatchNorm2d(d*2)
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1, dtype=torch.float32) # 4x4 
        self.conv3_bn = nn.BatchNorm2d(d*4)
        self.conv4 = nn.Conv2d(d*4, hidden_layer_size, 4, 1, 0, dtype=torch.float32) # 1x1 


    def forward(self, input):
        x = F.leaky_relu(self.conv1(input), 0.2)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = torch.sigmoid(self.conv4(x))

        return x
        

class ImageVAE_dec(nn.Module):
    def __init__(self, d=32, hidden_layer_size=100):
        super(ImageVAE_dec, self).__init__()

        # deconv back to normal image size 
        # starts as 1x1 img with hidden_layer_size channels 
        self.deconv1 = nn.ConvTranspose2d(hidden_layer_size, d*4, 4, 1, 0, dtype=torch.float32) # 4x4 
        self.deconv1_bn = nn.BatchNorm2d(d*4)
        self.deconv2 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1, dtype=torch.float32) # 8x8 
        self.deconv2_bn = nn.BatchNorm2d(d*2)
        self.deconv3 = nn.ConvTranspose2d(d*2, d, 4, 2, 1, dtype=torch.float32) # 16x16 
        self.deconv3_bn = nn.BatchNorm2d(d)
        self.deconv4 = nn.ConvTranspose2d(d, 1, 4, 2, 3, dtype=torch.float32) # 28x28 

        # NOTE: in deconv, the kernel size is important, https://distill.pub/2016/deconv-checkerboard/ 
        

    def forward(self, input):
        x = F.relu(self.deconv1_bn(self.deconv1(input)))
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        x = torch.tanh(self.deconv4(x))

        return x
    


In [ ]:
enc = ImageVAE_enc().to(device) 
dec = ImageVAE_dec().to(device) 


# training loop 
epoch = 1 
batch_size = 25
loss_fn = nn.MSELoss() 
optimizer = torch.optim.Adam([enc.parameters(), dec.parameters()], lr=1e-4) 

prev_avg_loss = 10000000 
all_avg_losses = [] 
while True: 
    epoch_losses = [] 
    for b in range(0, len(train_ds), batch_size): 
        imgs = train_ds[b:b+batch_size] 
        imgs = train_preprocess(imgs) 
        
        emb = enc(imgs) 
        rec = dec(emb) 
        # compare reconstruction loss 
        loss = loss_fn(imgs, rec) 
        
        epoch_losses.append(loss.item()) 

        loss.backward() 
        optimizer.step() 
        optimizer.zero_grad() 

    epoch_avg_loss = sum(epoch_losses)/len(epoch_losses) 
    print("EPOCH {} AVG LOSS: {}".format(epoch, epoch_avg_loss))

    all_avg_losses.append(epoch_avg_loss) 
    if epoch_avg_loss > 1.05*prev_avg_loss: 
        # assume it's converged already so cut 
        break 

    prev_avg_loss = epoch_avg_loss 

    epoch += 1 



In [ ]:
import matplotlib.pyplot as plt 

plt.plot(list(range(1, 1+len(all_avg_losses))), all_avg_losses) 
plt.xlabel('Epoch') 
plt.ylabel("MSELoss") 

plt.show() 